In [ ]:
def load_sps(resolution, resolution_wave, zred):
    """Read the SSPs into a CSPSpecBasis object.
    
    Also smooth all the models to the instrumental resolution of the 
    
    """
    
    t0 = time.time()
    print('Initializing CSPSpecBasis...')
    sps = CSPSpecBasis(zcontinuous=1, compute_vega_mags=False, vactoair_flag=False)
    print('...took {:.1f} seconds.'.format(time.time() - t0)) 
    
    sps.csp.params['smooth_lsf'] = True
    sps.csp.set_lsf(wave, delta_v)


    restwave = resolution_wave / (1 + zred)
    delta_v = 2.998e5 * resolution / restwave
    
def get_lsf(spec, miles_fwhm_aa=2.54, zred=0.0, **extras):
    """This method takes a spec file and returns the quadrature difference
    between the instrumental dispersion and the MILES dispersion, in km/s, as a
    function of wavelength
    """
    lightspeed = 2.998e5  # km/s
    # Get the SDSS instrumental resolution for this plate/mjd/fiber
    wave_obs = 10**spec['loglam']  # observed frame wavelength
    # This is the instrumental velocity resolution in the observed frame
    sigma_v = np.log(10) * lightspeed * 1e-4 * spec['wdisp']
    # filter out some places where sdss reports zero dispersion
    good = sigma_v > 0
    wave_obs, sigma_v = wave_obs[good], sigma_v[good]
    # Get the miles velocity resolution function at the corresponding
    # *rest-frame* wavelength
    wave_rest = wave_obs / (1 + zred)
    sigma_v_miles = lightspeed * miles_fwhm_aa / 2.355 / wave_rest

    # Get the quadrature difference
    # (Zero and negative values are skipped by FSPS)
    dsv = np.sqrt(np.clip(sigma_v**2 - sigma_v_miles**2, 0, np.inf))
    # Restrict to regions where MILES is used
    good = (wave_rest > 3525.0) & (wave_rest < 7500)

    # return the broadening of the rest-frame library spectra required to match
    # the obserrved frame instrumental lsf
    return wave_rest[good], dsv[good]
    
    
    
    sps.ssp.params['smooth_lsf'] = True
    sps.ssp.set_lsf(restwave, delta_v)
    
    return sps

set_lsf takes the restframe wavelength.  It is confusing when dealing with non-zero redshift.  IF the prism resolution is not great you might want to go the route of just using he BaSeL libraries throughout (by changing sps_vars and recompiling fsps/python-fsps).  Then the model resolution is all ~ the same over the whole wavelength region and, because it’s a ~constant velocity resolution instead of a constant delta(lam), is invariant with redshift.

By calling set_lsf and setting smooth_lsf to True the SSPs will be broadened (presumably to the instrumental resolution).  If you want additional broadening, e.g. from physical velocity dispersion, then you also need smoothtype and sigma_smooth in model_params.  But if the instrumental resolution is so poor that you don’t care about the physical velocity dispersion then you can set sigma_smooth to zero and no smoothing happens.

I should have said if sigma_smooth is zero then no *extra* smoothing is done beyond what is done to the SSPs by smooth_lsf=True

also I see some confusing bugs in the gist (I copied from a too old file).  Here’s a fixed get_lsf.  Note that this takes an input redshift and returns *wave_rest*, which is what set_lsf expects.  If I were you I’d switch to all BaSeL and replace sigma_v_miles with sigma_v_basel = 500, and remove the last restriction to 3525 < wave_rest < 7500

In [ ]:
def get_lsf(spec, miles_fwhm_aa=2.54, zred=0.0, **extras):
    """This method takes a spec file and returns the quadrature difference
    between the instrumental dispersion and the MILES dispersion, in km/s, as a
    function of wavelength
    """
    lightspeed = 2.998e5  # km/s
    # Get the SDSS instrumental resolution for this plate/mjd/fiber
    wave_obs = 10**spec['loglam']  # observed frame wavelength
    # This is the instrumental velocity resolution in the observed frame
    sigma_v = np.log(10) * lightspeed * 1e-4 * spec['wdisp']
    # filter out some places where sdss reports zero dispersion
    good = sigma_v > 0
    wave_obs, sigma_v = wave_obs[good], sigma_v[good]
    # Get the miles velocity resolution function at the corresponding
    # *rest-frame* wavelength
    wave_rest = wave_obs / (1 + zred)
    sigma_v_miles = lightspeed * miles_fwhm_aa / 2.355 / wave_rest

    # Get the quadrature difference
    # (Zero and negative values are skipped by FSPS)
    dsv = np.sqrt(np.clip(sigma_v**2 - sigma_v_miles**2, 0, np.inf))
    # Restrict to regions where MILES is used
    good = (wave_rest > 3525.0) & (wave_rest < 7500)

    # return the broadening of the rest-frame library spectra required to match
    # the obserrved frame instrumental lsf
    return wave_rest[good], dsv[good]

In [ ]:
sps.csp.params['smooth_lsf'] = True
sps.csp.set_lsf(wave, delta_v)

and actually looking at your code I see you’re not taking quadtrature difference from MILEs - I guess the Prism is so much lower resolution it doesn’t matter what MILES is.

I’m also a little confused about the redshifting. Assuming `resolution` in your load_sps is in dlam, I think you might actually want 

delta_v = c * resolution / resolution_wave
sps.csp.set_lsf(restwave, delta_v)

I.e. get the velocity resolution curve in the observed frame and then just shift it into the restframe.  But I’m not totally sure….need more coffee
